In [1]:
import requests
import bs4

In [2]:
base_url=input('Enter the wikipedia link from which you want to train your bot: ')
r = requests.get(base_url)
if (r.status_code == 200):
  print('Found it')
else:
  print('Could not find it')

Found it


In [3]:
soup = bs4.BeautifulSoup(r.text,'html.parser')

In [4]:
headers = []
for url in soup.findAll("h3"):
    headers.append(url.text)
    #print(url.text)

In [5]:
i = len(headers) - 1
counter = 0
while counter <= i:
    if headers[counter].startswith('\n'):
        headers.pop(counter)
        counter -= 1
    counter += 1
    i = len(headers) -1
#print(headers)

In [6]:
r = requests.get(base_url)
all_para = ""
soup = bs4.BeautifulSoup(r.text,'html.parser')
for iteri in range(len(headers)):
    deet = soup.find('h3', text = headers[iteri]) # Search for div tags of class 'entry-content content'
    for para in deet.find_next_siblings(): # Within these tags, find all p tags
        if para.name == "h2" or para.name == "h3":
            break
        elif para.name == "p":
            all_para += para.get_text()
            all_para += '\n'

In [7]:
with open('./wiki.txt', 'wb') as file_handler:
        file_handler.write(all_para.encode('utf8'))

In [8]:
import nltk # to process text data
import numpy as np # to represent corpus as arrays
import random 
import string # to process standard python strings
from sklearn.metrics.pairwise import cosine_similarity # We will use this later to decide how similar two sentences are
from sklearn.feature_extraction.text import TfidfVectorizer # Remember when you built a function to create a tfidf bag of words in Experience 2? This function does the same thing!

In [9]:
#nltk.download('punkt')
#nltk.download('wordnet')

In [10]:
filepath='wiki.txt'
corpus=open(filepath,'r',errors = 'ignore')
raw_data=corpus.read()
#print(raw_data)

In [11]:
raw_data=raw_data.lower() 
#print(raw_data)

In [12]:
sent_tokens=nltk.sent_tokenize(raw_data) 
#print(sent_tokens)

In [13]:
print("printing lists in new line") 
print(*sent_tokens, sep = "\n")

printing lists in new line
in 1971, george lucas wanted to film an adaptation of the flash gordon serial, but could not obtain the rights, so he began developing his own space opera.
[43][d] after directing american graffiti (1973), he wrote a two-page synopsis, which 20th century fox decided to invest in.
[44][45] by 1974, he had expanded the story into the first draft of a screenplay.
[46] the 1977 movie's success led lucas to make it the basis of an elaborate film serial.
[47] with the backstory he created for the sequel, lucas decided that the series would be a trilogy of trilogies.
[48] most of the main cast would return for the two additional installments of the original trilogy, which were self-financed by lucasfilm.
star wars was released on may 25, 1977, and first subtitled episode iv: a new hope in the 1979 book the art of star wars.
[49] episode v: the empire strikes back was released on may 21, 1980, also achieving wide financial and critical success.
the final film in the 

In [14]:
word_tokens=nltk.word_tokenize(raw_data)
#print(word_tokens)

In [15]:
#print(len(sent_tokens), len(word_tokens))

In [16]:
lemmer = nltk.stem.WordNetLemmatizer() #Initiate lemmer class. WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

In [17]:
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict))) #see previous section 1.2.5 lemmatization

In [18]:
GREETING_INPUTS = ["hello", "hi", "greetings", "sup", "what's up","hey", "hey there"]
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

In [19]:
def greeting(sentence):
    for word in sentence.split(): # Looks at each word in your sentence
        if word.lower() in GREETING_INPUTS: # checks if the word matches a GREETING_INPUT
            return random.choice(GREETING_RESPONSES) # replies with a GREETING_RESPONSE

In [20]:
def response(user_response):
    
    robo_response='' # initialize a variable to contain string
    sent_tokens.append(user_response) #add user response to sent_tokens
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english') 
    tfidf = TfidfVec.fit_transform(sent_tokens) #get tfidf value
    vals = cosine_similarity(tfidf[-1], tfidf) #get cosine similarity value
    idx=vals.argsort()[0][-2] 
    flat = vals.flatten() 
    flat.sort() #sort in ascending order
    req_tfidf = flat[-2] 
    
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [21]:
import datetime

def tell_time(sentence):
    for word in sentence.split():
        # your code here
            currentdt = datetime.datetime.now()
            return currentdt.strftime("%Y-%m-%d %H:%M:%S")

tell_time('time')

'2021-06-30 08:45:41'

In [ ]:
flag=True
print("CATHY: My name is Cathy. I will answer your queries. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("CATHY: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("CATHY: "+greeting(user_response))
            else:
                print("CATHY: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("CATHY: Bye! take care..")